In [1]:
import gurobipy as gp

In [2]:
# Parâmetros do problema
qtd_fabricas = 4
qtd_clientes = 9

# INSTÂNCIA 1 (Oferta maior que a demanda)
vet_ofertas = [140, 160, 200, 190]
vet_demandas = [50, 80, 30, 50, 100, 90, 20, 70, 120]

vet_custos = [[12, 25, 39, 17, 38, 40, 8, 25, 13],
              [17, 26, 20, 25, 30, 25, 14, 20, 15],
              [35, 15, 18, 20, 12, 42, 27, 26, 19],
              [28, 30, 37, 30, 28, 36, 16, 24, 32]]

oferta_total = sum(vet_ofertas)
demanda_total = sum(vet_demandas)
print(oferta_total, demanda_total)

690 610


In [3]:
# Rótulos das fábricas e clientes
fabricas = list()
for i in range(qtd_fabricas):
    fabricas.append("Fab_{}".format(i + 1))

clientes = list()
for j in range(qtd_clientes):
    clientes.append("Cli_{}".format(j + 1))

In [4]:
# Dicionários com as ofertas
ofertas = dict()
for idx, valor in enumerate(vet_ofertas):
    rotulo = fabricas[idx]
    ofertas[rotulo] = valor

# Dicionários com as demandas
demandas = dict()
for idx, valor in enumerate(vet_demandas):
    rotulo = clientes[idx]
    demandas[rotulo] = valor

In [5]:
custos = dict()
for i in range(qtd_fabricas):
    for j in range(qtd_clientes):
        rot_fab = fabricas[i]
        rot_cli = clientes[j]
        custos[rot_fab, rot_cli] = vet_custos[i][j]

In [6]:
m = gp.Model()

# Variáveis de decisão
x = m.addVars(fabricas, clientes, vtype=gp.GRB.INTEGER)

# Função objetivo
m.setObjective(
    gp.quicksum(x[i, j] * custos[i, j] for i in fabricas for j in clientes),
    sense=gp.GRB.MINIMIZE)

# Restrições de oferta
if oferta_total > demanda_total:
    c1 = m.addConstrs(
        gp.quicksum(x[i, j] for j in clientes) <= ofertas[i] for i in fabricas)
else:
    c1 = m.addConstrs(
        gp.quicksum(x[i, j] for j in clientes) == ofertas[i] for i in fabricas)

# Restrições de demanda
if demanda_total > oferta_total:
    c2 = m.addConstrs(
        gp.quicksum(x[i, j] for i in fabricas) <= demandas[j] for j in clientes)
else:
    c2 = m.addConstrs(
        gp.quicksum(x[i, j] for i in fabricas) == demandas[j] for j in clientes)

# Executa o modelo
m.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 13 rows, 36 columns and 72 nonzeros
Model fingerprint: 0xa70d03da
Variable types: 0 continuous, 36 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e+00, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+02]
Found heuristic solution: objective 13740.000000
Presolve time: 0.02s
Presolved: 13 rows, 36 columns, 72 nonzeros
Variable types: 0 continuous, 36 integer (0 binary)
Found heuristic solution: objective 13669.000000

Root relaxation: objective 1.060000e+04, 13 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0             

In [7]:
# Imprime o plano de transporte das fábricas
for i in fabricas:
    print("Origem:", i)
    for j in clientes:
        qtd = round(x[i, j].X)
        if qtd > 0:
            print("Transportar {} unidades para {}".format(qtd, j))
    print("")

Origem: Fab_1
Transportar 50 unidades para Cli_1
Transportar 30 unidades para Cli_4
Transportar 60 unidades para Cli_9

Origem: Fab_2
Transportar 10 unidades para Cli_3
Transportar 90 unidades para Cli_6
Transportar 60 unidades para Cli_9

Origem: Fab_3
Transportar 80 unidades para Cli_2
Transportar 20 unidades para Cli_3
Transportar 100 unidades para Cli_5

Origem: Fab_4
Transportar 20 unidades para Cli_4
Transportar 20 unidades para Cli_7
Transportar 70 unidades para Cli_8



In [8]:
# Relatório de oferta ou demanda desbalanceada
if oferta_total > demanda_total:
    print("As fábricas a seguir tem capacidade excedente:")
    for i in fabricas:
        sobra = round(c1[i].Slack)
        if sobra > 0:
            print("Fábrica:", i, sobra, "unidades")
elif demanda_total > oferta_total:
    print("Os clientes a seguir não tiveram toda a demanda atendida:")
    for j in clientes:
        sobra = round(c2[j].Slack)
        if sobra > 0:
            print("Cliente:", j, sobra, "unidades")
else:
    print("A oferta e demanda estão balanceadas")

As fábricas a seguir tem capacidade excedente:
Fábrica: Fab_4 80 unidades
